In [1]:
using CSV
using DataFrames
using Dates
using Distributions
using Serialization
using LinearAlgebra: diagm

In [2]:
projectbasepath = "../../covid-resource-allocation/"
include(joinpath(projectbasepath, "src/processing/LocalDataCommon.jl"));
include(joinpath(projectbasepath, "src/processing/GeographicData.jl"));

In [3]:
data_dir = "../data/";
outputdatapath = "../data/";

In [4]:
data_date = "2020_11_19";

In [5]:
SCENARIOS = [:moderate, :pessimistic, :catastrophic];
BEDTYPES  = [:allbeds, :icu, :ward];

In [6]:
los_dist = (
    allbeds = Gamma(1.723, 4.154),
    ward = Gamma(1.825, 3.772),
    icu = Gamma(0.9875, 12.8990),
);

In [7]:
start_date = Date(2020, 3, 25);
end_date   = Date(2021, 6, 30);
date_range = collect(start_date : Day(1) : end_date);

In [8]:
hospitals = ["JHH", "SH", "BMC", "HCGH", "SMH"];
N = length(hospitals);

In [9]:
adj = BitArray(ones(N,N) - diagm(ones(N)));

In [10]:
dist_matrix = diagm(fill(Inf, N));

In [11]:
capacity_names_full = ["Base Capacity", "Ramp-Up Capacity", "Surge Capacity", "Maximum Capacity"];
capacity_names_abbrev = ["baselinecap", "rampupcap", "surgecap", "maxcap"];

In [12]:
function load_capacity_jhhs(hospitals, bedtype, capacity_levels=[:baseline])
    beds_data = DataFrame(CSV.File(joinpath(data_dir, "jhhs_beds.csv")))

    beds_dict = Dict(row.hospital => Dict(
        "icu_covid_baseline" => row.icu_covid_baseline,
        "ward_covid_baseline" => row.ward_covid_baseline,
        "allbeds_covid_baseline" => row.icu_covid_baseline + row.ward_covid_baseline,
        "icu_covid_rampup" => row.icu_covid_rampup,
        "ward_covid_rampup" => row.ward_covid_rampup,
        "allbeds_covid_rampup" => row.icu_covid_rampup + row.ward_covid_rampup,
        "icu_covid_surge" => row.icu_covid_surge,
        "ward_covid_surge" => row.ward_covid_surge,
        "allbeds_covid_surge" => row.icu_covid_surge + row.ward_covid_surge,
        "icu_covid_surgeplus" => row.icu_covid_surgeplus,
        "ward_covid_surgeplus" => row.ward_covid_surgeplus,
        "allbeds_covid_surgeplus" => row.icu_covid_surgeplus + row.ward_covid_surgeplus,
        "icu_covid_max" => row.icu_covid_max,
        "ward_covid_max" => row.ward_covid_max,
        "allbeds_covid_max" => row.icu_covid_max + row.ward_covid_max,
    ) for row in eachrow(beds_data))

    if capacity_levels isa Symbol
        capacity = [beds_dict[h]["$(bedtype)_covid_$(capacity_levels)"] for h in hospitals]
    elseif capacity_levels isa AbstractArray
        capacity = hcat([[beds_dict[h]["$(bedtype)_covid_$(l)"] for h in hospitals] for l in capacity_levels]...)
    else
        error("Invalid capacity_levels")
    end

    return capacity
end;

In [13]:
function load_forecast_jhhs(scenario, bedtype)
    @assert(bedtype in [:icu, :ward, :allbeds])
    @assert(scenario in [:optimistic, :moderate, :pessimistic, :catastrophic])
    
    date_dir = "forecasts-" * replace(string(data_date), "_" => "-")

    forecast_admitted = DataFrame(CSV.File(joinpath(data_dir, date_dir, "jhhs_forecast_admitted_$(bedtype)_$(data_date).csv")))
    forecast_active   = DataFrame(CSV.File(joinpath(data_dir, date_dir, "jhhs_forecast_active_$(bedtype)_$(data_date).csv")))
    forecast_admitted_dict = Dict(row.date => row for row in eachrow(forecast_admitted))
    forecast_active_dict   = Dict(row.date => row for row in eachrow(forecast_active))

    day0 = minimum(date_range) - Day(1)
    forecast_admitted = [forecast_admitted_dict[d][c]  for c in (hospitals .* ("_" * string(scenario))), d in date_range]
    forecast_active   = [forecast_active_dict[d][c] for c in (hospitals .* ("_" * string(scenario))), d in date_range]

    beds = load_capacity_jhhs(hospitals, bedtype, :surge)
    capacity = load_capacity_jhhs(hospitals, bedtype, [:baseline, :rampup, :surge, :max])

    data = (
        scenario = scenario,
        bedtype = bedtype,

        los_dist = los_dist[bedtype],

        active = forecast_active,
        admitted = forecast_admitted,

        beds = beds,
        capacity = capacity,
    )

    return data
end;

In [14]:
maindata = Dict()
for scenario in SCENARIOS, bedtype in BEDTYPES
    maindata[(scenario,bedtype)] = load_forecast_jhhs(scenario, bedtype)
end

In [15]:
hospital_positions = Dict(
    "JHH"  => (lat = 39.2961773, long = -76.5939447),
    "SMH"  => (lat = 38.9364687, long = -77.1091435),
    "HCGH" => (lat = 39.2136187, long = -76.885917),
    "BMC"  => (lat = 39.290101,  long = -76.5468383),
    "SH"   => (lat = 38.9973285, long = -77.1105309),
);

In [16]:
completedata = (
    location_names = hospitals,
    location_names_short = hospitals,
    start_date = start_date,
    end_date = end_date,
    counties = nothing,
    states = nothing,
    dist_matrix = dist_matrix,
    locations_latlong = hospital_positions,
    casesdata = maindata,
);

In [17]:
serialize(joinpath(outputdatapath, "data_jhhs.jlser"), completedata);